<a href="https://colab.research.google.com/github/alonacode/AI/blob/main/My_practice_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [15]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [16]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [3]:
df.describe()


,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [4]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userId", "blog_id", "ratings"]], reader)

In [6]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [7]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [8]:
from surprise import SVD

alfo = SVD(n_factors=100,
           n_epochs=50,
           )

algo.fit(trainset)

Estimating biases using als...


In [9]:
from surprise import KNNBasic

algo = KNNBasic(k=10,
                min_k=1,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [10]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)

In [11]:
from surprise import BaselineOnly, accuracy


bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")


Estimating biases using als...
mae = 1.6466585423911964
mse = 3.470276480986576
rmse= 1.8628678109266303
fcp = 0.5044582564615132


In [12]:
algo.predict(uid=2,
             iid=97)

Prediction(uid=2, iid=97, r_ui=None, est=4.195301685778798, details={'was_impossible': False})

In [13]:
# Ініціалізація моделей
models = {
    "SVD": SVD(),
    "BaselineOnly": BaselineOnly(),
    "KNNBasic": KNNBasic()
}

# Тестування моделей
results = {}
for model_name, model in models.items():
    model.fit(trainset)  # Навчання
    predictions = model.test(testset)  # Тестування

    # Оцінка якості
    rmse = accuracy.rmse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)

    results[model_name] = {"RMSE": rmse, "MAE": mae}

# Виведення результатів
for model_name, metrics in results.items():
    print(f"{model_name}: RMSE = {metrics['RMSE']:.4f}, MAE = {metrics['MAE']:.4f}")


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
SVD: RMSE = 1.8407, MAE = 1.6371
BaselineOnly: RMSE = 1.7994, MAE = 1.6219
KNNBasic: RMSE = 1.8815, MAE = 1.6557


In [18]:
import optuna
def objective(trial):
    # Налаштування параметрів
    n_factors = trial.suggest_int("n_factors", 10, 100)
    n_epochs = trial.suggest_int("n_epochs", 10, 50)
    lr_all = trial.suggest_float("lr_all", 0.001, 0.1, log=True)
    reg_all = trial.suggest_float("reg_all", 0.01, 0.1, log=True)

    # Модель із параметрами
    model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)

    # Навчання та тестування
    model.fit(trainset)
    predictions = model.test(testset)

    # Використовуємо RMSE як функцію втрат
    return accuracy.rmse(predictions, verbose=False)

# Запуск Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Найкращі параметри
print("Найкращі параметри:", study.best_params)


[I 2024-11-27 18:57:57,991] A new study created in memory with name: no-name-08eb2999-19b0-49f8-a62b-ab98372d948a
[I 2024-11-27 18:58:01,301] Trial 0 finished with value: 1.9725712077679536 and parameters: {'n_factors': 56, 'n_epochs': 18, 'lr_all': 0.08613609242238343, 'reg_all': 0.014014447903159437}. Best is trial 0 with value: 1.9725712077679536.
[I 2024-11-27 18:58:05,067] Trial 1 finished with value: 1.8069468415493004 and parameters: {'n_factors': 69, 'n_epochs': 25, 'lr_all': 0.0021855948826807977, 'reg_all': 0.01583600715165771}. Best is trial 1 with value: 1.8069468415493004.
[I 2024-11-27 18:58:07,282] Trial 2 finished with value: 1.944803000016858 and parameters: {'n_factors': 69, 'n_epochs': 12, 'lr_all': 0.020702181833510084, 'reg_all': 0.018352497565047393}. Best is trial 1 with value: 1.8069468415493004.
[I 2024-11-27 18:58:14,238] Trial 3 finished with value: 1.8075748406034995 and parameters: {'n_factors': 54, 'n_epochs': 44, 'lr_all': 0.001546870374825393, 'reg_all':

Найкращі параметри: {'n_factors': 11, 'n_epochs': 10, 'lr_all': 0.001519810004083418, 'reg_all': 0.017606066508612093}


In [19]:
# Створюємо модель із найкращими параметрами
best_params = study.best_params
best_model = SVD(**best_params)

# Навчання та тестування
best_model.fit(trainset)
best_predictions = best_model.test(testset)

# Фінальна оцінка
final_rmse = accuracy.rmse(best_predictions)
final_mae = accuracy.mae(best_predictions)

print(f"Фінальний RMSE: {final_rmse:.4f}")
print(f"Фінальний MAE: {final_mae:.4f}")


RMSE: 1.7805
MAE:  1.6171
Фінальний RMSE: 1.7805
Фінальний MAE: 1.6171


In [20]:
from optuna.visualization import plot_param_importances

# Створення та відображення графіка
fig = plot_param_importances(study)
fig.show()


In [21]:
from optuna.visualization import plot_param_importances

# Створення та відображення графіка
fig = plot_param_importances(study)
fig.show()


In [22]:
from optuna.visualization import plot_slice

# Створення та відображення графіка
fig = plot_slice(study)
fig.show()
